In [ ]:
## Model: CNN (Convolutional Neural Network)

In [ ]:
from keras.models import Sequential
from keras import layers
from src.custom_loss import conv_loss

In [ ]:
feature_transformer = SlidingWindow(window=128, offset_percent=0.75)
disp.obj(feature_transformer, label='Feature Transformer')

In [ ]:
model_id = 'cnn'
kernel_size = 3
# optimizer=RMSprop(lr=1e-4)
optimizer='rmsprop'
strides=1

# TODO: Convert to a class so we have access to the model and the compiler
# noinspection PyShadowingNames
def compile_model(model, optimizer=optimizer, loss='binary_crossentropy'):
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['acc'],
                  run_eagerly=True)
    return model


# noinspection PyShadowingNames
def create_model(input_shape,
                 optimizer=optimizer,
                 loss='binary_crossentropy',
                 kernel_size=3,
                 strides=1,
                 nn_activation='relu'):
    model = Sequential()
    model.add(layers.Conv1D(32, kernel_size, activation=nn_activation, strides=strides, input_shape=input_shape))
    model.add(layers.AveragePooling1D(24))
    model.add(layers.Conv1D(64, kernel_size, activation=nn_activation, strides=strides))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(1, activation='sigmoid'))

    model.summary()

    # pass run_eagerly=True if there are problems converting Tensors <-> numpy
    return compile_model(model, optimizer, loss)

# TODO: Investigate why custom conv_loss performs so badly
get_custom_objects().update({'conv_loss': conv_loss})
# m = create_model(loss=conv_loss)

_X_txfm= feature_transformer.fit_transform(X_train)
_X_txfm = _X_txfm.reshape(_X_txfm.shape + (1,))

INPUT_SHAPE = (_X_txfm.shape[1], _X_txfm.shape[2])

# Create Model
model = create_model(INPUT_SHAPE)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              # loss=conv_loss,
              metrics=['acc'],
              run_eagerly=True)

# Create Wrapper
wrapper = KerasModelWrapper(model_id, model, feature_transformer, Project.path.MODEL)


# Fitting the Model

history1 = wrapper.fit(X_train, y_train,
                  epochs=5,
                  batch_size=16,
                  validation_split=0.2)

wrapper.save()
wrapper_saved = wrapper.load(wrapper.get_current_wrapper_path())
wrapper.remove_old_versions(keep=3)
model_loaded = wrapper_saved.model

history2 = wrapper_saved.fit(X_train, y_train,
                        epochs=5,
                        batch_size=16,
                        validation_split=0.2)

acc1 = history1.history['acc']
val_acc1 = history1.history['val_acc']
loss1 = history1.history['loss']
val_loss1 = history1.history['val_loss']

acc2 = acc1 + history2.history['acc']
val_acc2 = val_acc1 + history2.history['val_acc']
loss2 = loss1 + history2.history['loss']
val_loss2 = val_loss1 + history2.history['val_loss']

epochs1 = range(len(acc1))
epochs2 = range(len(acc2))

plt.plot(epochs2, acc2, 'bo', label='Training acc2', color='green')
plt.plot(epochs2, val_acc2, 'b', label='Validation acc2', color='green')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs2, loss2, 'bo', label='Training loss2', color='green')
plt.plot(epochs2, val_loss2, 'b', label='Validation loss2', color='green')
plt.title('Training and validation loss')
plt.legend()

plt.show()